In [1]:
from pymongo import MongoClient
import pprint

import pandas as pd

import numpy as np

# Requests sends and recieves HTTP requests.
import requests

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup

import json
import time

In [2]:
client = MongoClient('localhost', 27017)
db_eur = client['eur_scraper']

rates = db_eur['rates']

In [3]:
url = 'https://www.ecb.europa.eu/stats/policy_and_exchange_rates/key_ecb_interest_rates/html/index.en.html'

In [4]:
r = requests.get(url)
r.status_code

200

In [5]:
soup = BeautifulSoup(r.text, 'html.parser')

In [6]:
links = soup.find_all('tr')

In [7]:
len(links)

53

In [8]:
links[4].find_all('td')

[<td class="number"><strong>2016</strong></td>,
 <td class="number">16 Mar.</td>,
 <td class="number">−0.40</td>,
 <td class="number">0.00</td>,
 <td class="number">-</td>,
 <td class="number">0.25</td>]

In [27]:
def scrape_eur(links):
    
    """
    PARAMETERS:
    links - a list of parsable html tags generated by BeautifulSoup, pertinent to the Reserve Bank of Australia's website
    
    RETURNS:
    None - uploads rows of rate data to MongoDB
    """

    for ind, link in enumerate(links):
        
        #create accumulator dictionary
        d = dict()
        
        # compile dates and rates
        for idx,i in enumerate(link.find_all('td')):

            if idx == 0:
                if i.text != '\xa0' and i.text != '':
                    d['year'] = i.text
                else:
                    count = 1
                    while i.text == '\xa0' or i.text == '':
                        i = links[ind-count].find_all('td')[0]
                        count += 1
                    d['year'] = i.text

            elif idx == 1:
                d['day_month'] = i.text
                    

            elif idx == 2:
                d['eur_rate'] = i.text

            else:
                continue

    #     rates.insert_one(d)

    #     time.sleep(1)

        print(d)
    return None
    

In [28]:
scrape_eur(links)

{}
{}
{'year': 'With effect from', 'day_month': '\xa0'}
{'year': '2019', 'day_month': '18 Sep.', 'eur_rate': '−0.50'}
{'year': '2016', 'day_month': '16 Mar.', 'eur_rate': '−0.40'}
{'year': '2015', 'day_month': '9 Dec.', 'eur_rate': '−0.30'}
{'year': '2014', 'day_month': '10 Sep.', 'eur_rate': '−0.20'}
{'year': '2014', 'day_month': '11 Jun.', 'eur_rate': '−0.10'}
{'year': '2013', 'day_month': '13 Nov.', 'eur_rate': '0.00'}
{'year': '2013', 'day_month': '8 May.', 'eur_rate': '0.00'}
{'year': '2012', 'day_month': '11 Jul.', 'eur_rate': '0.00'}
{'year': '2011', 'day_month': '14 Dec.', 'eur_rate': '0.25'}
{'year': '2011', 'day_month': '9 Nov.', 'eur_rate': '0.50'}
{'year': '2011', 'day_month': '13 Jul.', 'eur_rate': '0.75'}
{'year': '2011', 'day_month': '13 Apr.', 'eur_rate': '0.50'}
{'year': '2009', 'day_month': '13 May', 'eur_rate': '0.25'}
{'year': '2009', 'day_month': '8 Apr.', 'eur_rate': '0.25'}
{'year': '2009', 'day_month': '11 Mar.', 'eur_rate': '0.50'}
{'year': '2009', 'day_month':